In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import librosa
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

In [ ]:
def escalonarFeatures(caminhoCSV):
  # Função para escalonar as features
  # Essa função recebe o dataframe geral, remove a primeira e última colunas (nome e classificação) 
  # faz o escalonamento e depois coloca as colunas de nome e classificação de volta.

  dataframeGeral = pd.read_csv(caminhoCSV)

  # COPIANDO AS COLUNAS DE NOME E CLASSIFICACAO
  colunaArquivo       = dataframeGeral["nomeArquivo"]
  colunaClassificacao = dataframeGeral["classificacaoCorreta"]

  # DELETANDO AS COLUNAS ARQUIVO E CLASSIFICACAO
  dataframeGeral = dataframeGeral.drop(['nomeArquivo', 'classificacaoCorreta'], axis=1).to_numpy()

  # ESCALONANDO
  dataframeGeral = pd.DataFrame(StandardScaler().fit_transform(dataframeGeral))

  # COM ESSA COISA DE TIRA E POE COLUNA, O PANDAS NAO SABE LINDAR COM OS INDEXES,
  # ABAIXO EU ESTOU RESETANDO TUDO
  colunaArquivo.reset_index(inplace=True, drop=True)
  colunaClassificacao.reset_index(inplace=True, drop=True)
  dataframeGeral.reset_index(inplace=True, drop=True)

  # ADICIONANDO AS COLUNAS QUE FORAM EXCLUIDAS (posicaoNovaColuna, nomeNovaColuna, valorParaTodasAsLinhas)
  dataframeGeral.insert(0, "nomeArquivo", colunaArquivo, True)
  dataframeGeral.insert(len(dataframeGeral.columns), "classificacaoCorreta", colunaClassificacao, True)

  # SOBRESCREVENDO O CSV QUE JA EXISTE
  dataframeGeral.to_csv(caminhoCSV, index=False)

  # LIBERANDO MEMORIA SO PRA GARANTIR
  del dataframeGeral
  del colunaArquivo
  del colunaClassificacao

### Parâmetros Iniciais

In [ ]:
dirOrigem   = "/content/drive/My Drive/datasets/ESC_10_16kHz_8bits/original/"
caminhoCSV  = dirOrigem + "ESC-10_original_espectrogramas_150x150.csv"
horizontal  = 150
vertical    = 150
arquivos    = os.listdir(dirOrigem)
qtdArquivos = len(arquivos)

In [ ]:
aaa = []

for l in np.arange(0,horizontal*vertical):
  aaa.append(str(l))
colunasTop = ['nomeArquivo'] + aaa + ['classificacaoCorreta']

dataframe = pd.DataFrame(columns=colunasTop)
dataframe.to_csv(caminhoCSV, index=False)

In [ ]:
for i, arquivoAtual in enumerate(arquivos):
    if arquivoAtual[-4:] == ".wav":
        
        sinalOriginal, freqAmostragem = librosa.load(dirOrigem+arquivoAtual, sr=None, mono=True)
        
        mel_spec = librosa.feature.melspectrogram(y=sinalOriginal, sr=freqAmostragem, hop_length=int(len(sinalOriginal)/horizontal), n_mels=vertical)[:vertical,:horizontal]
        mel_db   = librosa.power_to_db(mel_spec)
        features = mel_db.flatten()

        assert len(features) == horizontal*vertical, "O arquivo " + arquivoAtual + " recebeu " + str(len(features)) + ". O correto seria ter recebido " + str(horizontal*vertical) + "."

        pd.DataFrame([[arquivoAtual] + list(features) + [arquivoAtual.split("_")[0]]]).to_csv(caminhoCSV, mode='a', header=False, index=False)

        print(str(100*((i+1)/qtdArquivos))[:5]+"%", end=" ")

0.248% 0.496% 0.744% 0.992% 1.240% 1.488% 1.736% 1.985% 2.233% 2.481% 2.729% 2.977% 3.225% 3.473% 3.722% 3.970% 4.218% 4.466% 4.714% 4.962% 5.210% 5.459% 5.707% 5.955% 6.203% 6.451% 6.699% 6.947% 7.196% 7.444% 7.692% 7.940% 8.188% 8.436% 8.684% 8.933% 9.181% 9.429% 9.677% 9.925% 10.17% 10.42% 10.66% 10.91% 11.16% 11.41% 11.66% 11.91% 12.15% 12.40% 12.65% 12.90% 13.15% 13.39% 13.64% 13.89% 14.14% 14.39% 14.64% 14.88% 15.13% 15.38% 15.63% 15.88% 16.12% 16.37% 16.62% 16.87% 17.12% 17.36% 17.61% 17.86% 18.11% 18.36% 18.61% 18.85% 19.10% 19.35% 19.60% 19.85% 20.09% 20.34% 20.59% 20.84% 21.09% 21.33% 21.58% 21.83% 22.08% 22.33% 22.58% 22.82% 23.07% 23.32% 23.57% 23.82% 24.06% 24.31% 24.56% 24.81% 25.06% 25.31% 25.55% 25.80% 26.05% 26.30% 26.55% 26.79% 27.04% 27.29% 27.54% 27.79% 28.03% 28.28% 28.53% 28.78% 29.03% 29.28% 29.52% 29.77% 30.02% 30.27% 30.52% 30.76% 31.01% 31.26% 31.51% 31.76% 32.00% 32.25% 32.50% 32.75% 33.00% 33.25% 33.49% 33.74% 33.99% 34.24% 34.49% 34.73% 34.98% 35.23% 35.48%

In [ ]:
escalonarFeatures(caminhoCSV)

In [ ]:
dataframe = pd.read_csv(caminhoCSV)
dataframe.head(5)

,nomeArquivo,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,...,22461,22462,22463,22464,22465,22466,22467,22468,22469,22470,22471,22472,22473,22474,22475,22476,22477,22478,22479,22480,22481,22482,22483,22484,22485,22486,22487,22488,22489,22490,22491,22492,22493,22494,22495,22496,22497,22498,22499,classificacaoCorreta
0,dog_155312A_3.wav,-0.941584,-0.650197,-0.323724,-0.263636,-0.446884,-0.856064,-0.920510,-0.789319,-0.783608,-0.830179,-0.815437,-0.843864,-1.012416,-1.317034,-1.512969,-1.302699,-1.109899,-1.002213,-0.942013,-0.792929,-0.585175,-0.538067,-0.671357,-0.799205,-0.819001,-0.859957,-0.964364,-1.130523,-1.227596,-1.293079,-1.381926,-1.104098,-0.958262,-1.029708,-1.300319,-1.494301,-1.456954,-1.115637,-0.692813,...,-0.533929,-0.448932,-0.132775,0.169345,0.065131,-0.286497,-0.292345,0.069455,0.197601,-0.016863,-0.157845,0.127806,0.154845,-0.176328,-0.310533,-0.279741,-0.209027,-0.208312,-0.260513,-0.547894,-0.797902,-0.531874,-0.170168,-0.153033,-0.287384,-0.231806,0.294206,0.646183,0.367284,0.084459,0.473204,0.800804,0.649419,-0.063064,-0.691771,-0.326481,0.139402,-0.099541,0.083308,dog
1,sneezing_82538A_2.wav,-0.013736,-0.011054,0.090659,0.243415,0.402149,0.560722,0.700495,0.809764,0.887429,0.942728,1.001547,1.039111,1.076284,1.087437,1.097044,1.104211,1.128217,1.135745,1.165330,1.215631,1.214405,1.206046,1.212877,1.194123,1.187937,1.175801,1.170267,1.118932,1.111213,1.094104,1.106413,1.101909,1.104846,1.125928,1.123194,1.091950,1.064341,1.027200,0.953611,...,0.422993,0.359546,0.272212,0.177952,0.171349,0.169186,0.160055,0.157737,0.190280,0.224454,0.222987,0.237557,0.172846,0.096549,-0.000082,-0.158135,-0.258841,-0.327225,-0.323430,-0.338223,-0.343040,-0.306481,-0.280502,-0.260030,-0.312255,-0.375909,-0.407164,-0.407470,-0.445819,-0.462267,-0.457923,-0.465046,-0.383276,-0.288261,-0.238923,-0.202741,-0.183853,-0.228268,-0.234753,sneezing
2,dog_30226A_1.wav,0.446846,0.260349,0.119102,0.132271,-0.029457,-0.251478,-0.157363,-0.014224,-0.130329,-0.414567,-0.217135,-0.000331,-0.147213,-0.616957,-0.291019,-0.205301,0.081460,-0.232082,-0.075100,-0.075062,-0.183512,0.159403,0.139883,-0.071237,0.076776,0.168116,0.478630,0.594998,0.508653,0.381976,-0.014398,0.006908,0.080735,0.169824,0.104293,0.154320,-0.077946,-0.434549,-0.014185,...,-0.586747,0.409487,0.314898,-0.249187,-0.590364,-0.420067,0.501855,0.196347,-0.046929,-0.098640,-0.186871,-0.433869,-0.311916,0.506689,0.380334,0.108139,0.888995,1.043123,-0.183697,-1.034753,-0.903446,-0.316067,0.872672,0.232901,0.363749,-0.231169,-0.867731,-0.618768,-0.333774,-0.161785,-0.567598,-0.399754,0.097285,0.154902,-0.422421,-0.039932,0.434189,-0.532251,-1.108988,dog
3,seawaves_167063B_4.wav,0.894024,0.456028,-0.204045,-0.557158,-0.199184,-0.169925,-0.622570,-0.574789,-0.290226,-0.488294,-0.701093,-0.451696,-0.687003,-0.661283,-0.395460,-0.305663,-0.584830,-0.804690,-0.887663,-0.694349,-0.361348,-0.440881,-0.578145,-0.525109,-0.547351,-0.648508,-0.764227,-0.738212,-0.763291,-0.374844,-0.457736,-1.178164,-1.217225,-0.367786,-0.053918,-0.220112,-0.256548,-0.188193,-0.218873,...,0.071050,-0.428122,-0.551877,0.131315,0.850801,-0.322225,-0.494396,0.221537,0.539591,0.232011,0.118898,-0.175566,-0.326452,0.614877,0.663936,0.500550,0.201276,0.544742,1.226950,1.106224,0.562705,0.540411,0.321762,0.835335,0.999231,0.907458,0.056664,0.118102,0.443619,0.755983,0.294017,0.323708,1.292528,0.873602,0.844154,0.512732,-0.937265,-1.147667,-0.353984,seawaves
4,seawaves_132157A_2.wav,0.215583,0.173342,0.342511,0.181587,-0.187067,-0.033053,0.363307,0.498337,0.549381,0.163079,-0.104785,-0.181287,-0.302856,0.133506,0.261827,0.237010,-0.030087,-0.260704,0.084661,-0.067612,-0.395486,-0.290013,-0.406394,-0.056461,-0.052932,-0.195405,-0.370725,-0.297387,-0.424839,-0.594621,-0.330200,-0.263803,-0.055621,-0.234165,-0.190364,-0.003715,-0.195388,-0.556559,-0.379667,...,1.136510,0.143461,-0.503368,-0.386327,-0.932546,-0.336976,0.182109,-0.511022,-0.527922,0.301895

In [ ]:
# sinalOriginal, freqAmostragem = librosa.load(dirOrigem+"sneezing_109505A_2.wav", sr=None, mono=True)        
# mel_spec = librosa.feature.melspectrogram(y=sinalOriginal, sr=freqAmostragem, hop_length=int(len(sinalOriginal)/horizontal), n_mels=vertical)[:vertical,:horizontal]
# mel_db   = librosa.power_to_db(mel_spec)

# print(mel_db.shape)

# plt.imshow(mel_db)